In [ ]:
from tifffile import imread
from tifffile import imwrite
from tifffile import imsave
from matplotlib import pyplot
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pickle
import os
from os import listdir
import itertools

In [ ]:
# check statistics of all the images in the folder
folder = 'folder_name'
for images in os.listdir(folder):
    if (images.endswith('.tif')):
        a = imread(os.path.join(folder,images))
        a_r_inf = np.where(a > 65530, 0, a)
        a_fl = a_r_inf.reshape(-1, *a_r_inf.shape[2:])
        locals()[os.path.splitext(images)[0]] = a_fl #modify the value of the given string through the local’s dictionary.
        print(os.path.splitext(images)[0]) #get filename
        print(a.shape)
        print(a_fl.shape)
        print(a_fl.dtype)
        for i in range(0, 8):
            print(stats.describe(a_fl[:,i]))

In [ ]:
# get the index of NoData pixels
NoData_p = np.where(Clip_PS_rp__20201115p[:,0] == 0)
NoData_w = np.where(Clip_PS_rp__20201115w[:,0] == 0)
boundary_p = NoData_p[0].tolist()
boundary_w = NoData_w[0].tolist()
print('P pixel number:', 13015-len(NoData_p[0])) # NoData_p[0] refers to index of values == 0 in axis 0
print('W pixel number:', 18669-len(NoData_w[0]))

In [ ]:
# create a list of variables of only PlanetScope (PS) images
keylist = list(locals().keys()) # get keys from all variables from local dic, locals() can be used to list all variables in memory. 
variable_list = []
for variables in keylist:
    if variables.startswith('Clip_PS_rp__'):
        variable_list.append(variables)
print(variable_list)
print('image number:', len(variable_list))

In [ ]:
# create NDSI for each PS variable
for variables in variable_list:
    df_NDSI_variables = pd.DataFrame()
    df_variables = pd.DataFrame(locals()[variables]).astype('float32') # locals()[variables] is for expressing the value of the variables
    df_variables.columns = ['costal_B','B','G1','G2','Y','R','R_edge','NIR']
    for a, b in itertools.combinations(df_variables.columns, 2):
        df_NDSI_variables[f'{a}-{b}/{a}+{b}'] = (df_variables[a]-df_variables[b]).div(df_variables[a]+df_variables[b])
    arr_NDSI_variables = df_NDSI_variables.to_numpy()
    arr_NDSI_variables_clean = np.nan_to_num(arr_NDSI_variables, nan=0)
    locals()['arr_{}'.format(variables)] = arr_NDSI_variables_clean
    print('df_NDSI_{}_shape:'.format(variables), df_NDSI_variables.shape)
    print('arr_NDSI_{}_shape:'.format(variables), arr_NDSI_variables_clean.shape)
    print('arr_NDSI_{}_dtypes:'.format(variables), arr_NDSI_variables_clean.dtype)
    for i in range(0, 28):
        print(stats.describe(arr_NDSI_variables_clean[:,i]))

In [ ]:
# import the trained model
PS_rf = pickle.load(open('file_name', 'rb'))

In [ ]:
# create a list of variables of only PS arrays
keylist1 = list(locals().keys()) # get keys from all variables in local, locals() can be used to list all variables in memory 
variable_list1 = []
for variables in keylist1:
    if variables.startswith('arr_Clip_PS_rp'):
        variable_list1.append(variables)
print(variable_list1)
print('array number:', len(variable_list1))

In [ ]:
# import data 
df_NDSI_2021 = pd.read_csv('file_name') 
df_sl_clean_2021 = pd.read_csv('file_name')
# train_test set splitting (stratified with day of the year and the vineyards)
X_2021, y_2021 = df_NDSI_2021.values, df_sl_clean_2021.iloc[:,0].values
X_train_2021, X_test_2021, y_train_2021, y_test_2021 = train_test_split(X_2021, y_2021, test_size=0.3, 
                                    random_state=5) 
# get standardization scaler
scaler_st = StandardScaler()
X_train_2021_st = scaler_st.fit_transform(X_train_2021)

In [ ]:
# create grapevine water status (GWS) map
for variables in variable_list1:
    raster_st = scaler_st.transform(locals()[variables]) # standardization
    PS_WS = PS_rf.predict(raster_st)
    print('GWS_min_{}:'.format(variables),np.min(PS_WS))
    print('GWS_max_{}:'.format(variables),np.max(PS_WS))
    if PS_WS.shape[0] == 13015:
        for i in boundary_p:
            PS_WS[i] = np.nan # convert all pixels not in the study area into nan
        PS_WS_rs = PS_WS.reshape(95, 137) 
    elif PS_WS.shape[0] == 18669:
        for i in boundary_w:
            PS_WS[i] = np.nan
        PS_WS_rs = PS_WS.reshape(127, 147)
    locals()['PS_cb_{}'.format(variables)] = PS_WS_rs

In [ ]:
# create a list of GWS array
keylist2 = list(locals().keys()) # get keys from all variables in local, locals() can be used to list all variables in memory 
variable_list2 = []
for variables in keylist2:
    if variables.startswith('PS_cb_arr_Clip_PS_rp'):
        variable_list2.append(variables)
print(variable_list2)
print('GWS array number:', len(variable_list2))

In [ ]:
# export images
for variables in variable_list2:
    imwrite('path'.format(variables), locals()[variables], resolution=(1./3., 1./3.))
# add planarconfig='contig' for saving multibands